In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#https://markets.businessinsider.com/news/nvda-stock?p={page}  ->> Nvidia stock news
#https://markets.businessinsider.com/news/amzn-stock?p={page}  ->> Amazon stock news
#https://markets.businessinsider.com/news/tsla-stock?p={page}   ->> Tesla stock news
#https://markets.businessinsider.com/news/googl-stock?p={page}  ->> Google stock news
#https://markets.businessinsider.com/news/adbe-stock?p={page} ->> Adobe stock news
#https://markets.businessinsider.com/news/axp-stock?p={page}  ->> American Express stock news
#https://markets.businessinsider.com/news/meta-stock?p={page}->> Meta stock news
#https://markets.businessinsider.com/news/spot-stock?p={page}->> Spotify stock news
#https://markets.businessinsider.com/news/msft-stock?p={page}     ->> Microsoft stock news
#https://markets.businessinsider.com/news/jpm-stock?p={page}  -->JPMorgan stock news

## Imports

In [3]:
!pip install -q yfinance

from bs4 import BeautifulSoup
import requests
import pandas as pd
import yfinance as yf
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Fetcthing data

In [4]:
def fetch_stock_data(ticker):
    print("Fetching stock prices...")
    stock_data = yf.download(ticker, period = "max")
    stock_data = stock_data[['Close']]
    return stock_data

def fetch_news(ticker):
    columns = ['datetime','ticker','source', 'headline' ]
    df = pd.DataFrame(columns=columns)
    counter = 0
    ticker = str(ticker).lower()
    print("Fetching news headlines...")
    for page in tqdm(range(1,200)):
        url = f'https://markets.businessinsider.com/news/{ticker}-stock?p={page}'
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup (html, 'lxml')
        articles = soup.find_all('div', class_ = 'latest-news__story')
        for article in articles:
            datetime = article.find('time', class_ = 'latest-news__date').get('datetime')
            title = article.find('a', class_ = 'news-link').text
            source = article.find('span', class_ = 'latest-news__source').text
        #   link = article.find('a', class_ = 'news-link').get('href')
            ticker = 'NVDA'       # 'AMZN', 'TSLA', 'GOOGL', 'ADBE', 'AXP', 'META', 'NVDA' ,'SPOT' ,'MSFT' ,'JPM'
            df = pd.concat([pd.DataFrame([[datetime,ticker, source,title]], columns=df.columns), df], ignore_index=True)
            counter += 1
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    df.drop(columns=['datetime'], inplace=True)
    print (f'{counter} headlines scraped from {page+1} pages')
    return df

# Sentiment Analysis

In [29]:
def calculate_sentiment_scores(headlines_df):
    headlines_df['sentiment'] = headlines_df['headline'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
    headlines_df['date'] = pd.to_datetime(headlines_df['date'])
    
    return headlines_df

def aggregate_sentiment_scores(headlines_df):
    sentiment_summary = headlines_df.groupby('date')['sentiment'].mean()
    return sentiment_summary

def generate_trading_signals(sentiment_summary):
    signals = sentiment_summary.apply(lambda x: 1 if x > 0.2 else (-1 if x < -0.2 else 0))
    return signals

# Plot and calculations

In [30]:
def calculate_portfolio_metrics(portfolio):
    total_trades = len(portfolio) // 2
    wins = portfolio[portfolio['type'] == 'sell']['profit'] > 0
    win_percentage = wins.mean() * 100
    total_profit = portfolio[portfolio['type'] == 'sell']['profit'].sum()
    
    return total_trades, win_percentage, total_profit

def calculate_sharpe_ratio(portfolio, risk_free_rate=0.01):
    daily_returns = portfolio[portfolio['type'] == 'sell']['profit']
    excess_returns = daily_returns - risk_free_rate
    sharpe_ratio = excess_returns.mean() / excess_returns.std()
    return sharpe_ratio

def calculate_max_drawdown(portfolio):
    portfolio['cumulative_profit'] = portfolio['profit'].cumsum()
    cumulative_max = portfolio['cumulative_profit'].cummax()
    drawdown = portfolio['cumulative_profit'] - cumulative_max
    max_drawdown = drawdown.min()
    return max_drawdown

import plotly.graph_objects as go

def plot_signals(stock_data, portfolio, ticker):
    start_date = portfolio['date'].min()
    end_date = portfolio['date'].max()

    # Filter stock data to the date range available in the portfolio
    stock_data = stock_data[(stock_data.index >= start_date) & (stock_data.index <= end_date)]
    
    buy_signals = portfolio[portfolio['type'] == 'buy']
    sell_signals = portfolio[portfolio['type'] == 'sell']
    fig = go.Figure()
    # Add stock price trace
    fig.add_trace(go.Scatter(
        x=stock_data.index, 
        y=stock_data['Close'],
        mode='lines',
        name='Stock Price',
        line=dict(color='blue')
    ))

    # Add buy signals
    fig.add_trace(go.Scatter(
        x=buy_signals['date'], 
        y=buy_signals['price'],
        mode='markers',
        name='Buy Signal',
        marker=dict(symbol='triangle-up', color='green', size=10)
    ))

    # Add sell signals
    fig.add_trace(go.Scatter(
        x=sell_signals['date'], 
        y=sell_signals['price'],
        mode='markers',
        name='Sell Signal',
        marker=dict(symbol='triangle-down', color='red', size=10)
    ))
    # Update layout for better presentation
    fig.update_layout(
        title=f'Stock Price with Buy and Sell Signals for {ticker}',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend',
        hovermode='x'
    )
    fig.show()


# Simulating Trades

In [31]:
# def simulate_trades(stock_data, trading_signals):
#     portfolio = []
#     position = 0
#     buy_price = 0
#     for date, price in (stock_data['Close'].items()):
#         if date in trading_signals.index:
#             signal = trading_signals.loc[date]
#             if signal == 1 and position == 0:  # Buy signal
#                 position = 1
#                 buy_price = price
#                 portfolio.append({"date": date, "type": "buy", "price": buy_price  })
#             elif signal == -1 and position == 1and (price>buy_price) :  # Sell signal
#                 position = 0
#                 sell_price = price
#                 profit = sell_price - buy_price
#                 portfolio.append({"date": date, "type": "sell", "price": sell_price, "profit": profit})

#     return pd.DataFrame(portfolio)


# def final(ticker, rate):

#     # Fetch stock data
#     stock_data = fetch_stock_data(ticker)

#     # Fetch news data
#     news_data = fetch_news(ticker)

#     # Calculate sentiment scores
#     news_data_with_scores = calculate_sentiment_scores(news_data)

#     # Aggregate sentiment scores by date
#     sentiment_summary = aggregate_sentiment_scores(news_data_with_scores)

#     # Generate trading signals
#     trading_signals = generate_trading_signals(sentiment_summary)

#     # Simulate trades
#     print("Simulating trades...")
#     portfolio = simulate_trades(stock_data, trading_signals)

#     # Calculate portfolio metrics
#     total_trades, win_percentage, total_profit = calculate_portfolio_metrics(portfolio)

#     # Print the portfolio
#     print(portfolio)
#     print(f"\nTotal Trades: {total_trades}")
#     print(f"Win Percentage: {win_percentage:.2f}%")
#     print(f"Total Portfolio Returns: ${total_profit:.2f}")
#     print(f"Sharpe ratio:{calculate_sharpe_ratio(portfolio, rate)}")
#     print(f"Max drawdown: {calculate_max_drawdown(portfolio)}")
#     # Plot buy and sell signals
#     plot_signals(stock_data, portfolio, ticker)

In [32]:
def simulate_trades(stock_data, trading_signals,capital):
    portfolio = []
    position = 0
    buy_price = 0
    quantity =0
    for date, price in (stock_data['Close'].items()):
        if date in trading_signals.index:
            signal = trading_signals.loc[date]
            if signal == 1 and position == 0:  # Buy signal
                position = 1
                buy_price = price
                quantity = capital/price
                capital = capital%price
                portfolio.append({"date": date, "type": "buy", "price": buy_price, "capital":capital})
            elif signal == -1 and position == 1 and (price>buy_price) :  # Sell signal
                position = 0
                sell_price = price
                profit = (sell_price - buy_price)*quantity
                capital = capital+ quantity*sell_price
                portfolio.append({"date": date, "type": "sell", "price": sell_price,"capital":capital, "profit": profit})

    return pd.DataFrame(portfolio)


def final(ticker, rate, intial, df):

    # Fetch stock data
    stock_data = fetch_stock_data(ticker)

    # Fetch news data
    news_data = df
    # Calculate sentiment scores
    news_data_with_scores = calculate_sentiment_scores(news_data)
    # Aggregate sentiment scores by date
    sentiment_summary = aggregate_sentiment_scores(news_data_with_scores)
    # Generate trading signals
    trading_signals = generate_trading_signals(sentiment_summary)
    # Simulate trades
    print("\nSimulating trades...")
    portfolio = simulate_trades(stock_data, trading_signals, intial)
    # Calculate portfolio metrics
    total_trades, win_percentage, total_profit = calculate_portfolio_metrics(portfolio)
    # Print the portfolio
    print(portfolio)
    print(f"\nInitial capital: ${intial}")
    print(f"Total Trades: {total_trades}")
    print(f"Win Percentage: {win_percentage:.2f}%")
    print(f"Total Portfolio Returns: ${total_profit:.2f}")
    print(f"Sharpe ratio:{calculate_sharpe_ratio(portfolio, rate):.2f} with risk free rate of {rate}")
    print(f"Max drawdown: {calculate_max_drawdown(portfolio)}")
    # Plot buy and sell signals
    plot_signals(stock_data, portfolio, ticker)

In [33]:
# news = fetch_news("MSFT")

In [34]:
final("MSFT", 0.03, 10000, news)

Fetching stock prices...


[*********************100%%**********************]  1 of 1 completed



Simulating trades...
         date  type       price       capital       profit
0  2016-10-14   buy   57.419998      8.920319          NaN
1  2016-12-29  sell   62.900002  10963.292233   954.371914
2  2017-02-27   buy   64.230003     44.191662          NaN
3  2017-04-24  sell   67.529999  11570.753587   563.269692
4  2017-04-25   buy   67.919998     24.353898          NaN
5  2017-06-15  sell   69.900002  11932.418020   337.310535
6  2017-06-16   buy   70.000000     32.418020          NaN
7  2017-07-31  sell   72.699997  12425.085929   460.249889
8  2017-08-01   buy   72.580002     13.905616          NaN
9  2017-08-08  sell   72.790001  12474.941622    35.950077
10 2017-08-10   buy   71.410004     49.600985          NaN
11 2017-09-07  sell   74.339996  13036.396489   511.853882
12 2017-09-18   buy   75.160004     33.715855          NaN
13 2017-10-09  sell   76.290001  13266.108772   195.996428
14 2017-10-23   buy   78.830002     22.668465          NaN
15 2017-10-24  sell   78.860001  1

## Testing for Nvidia stocks

In [35]:
# nvdia_news = fetch_news("NVDA")

In [36]:
final("NVDA", 0.01, 1000, nvdia_news)

Fetching stock prices...


[*********************100%%**********************]  1 of 1 completed



Simulating trades...
         date  type      price       capital       profit
0  2016-10-14   buy   1.649750      0.251503          NaN
1  2016-12-29  sell   2.785750   1688.840636   688.589133
2  2017-02-27   buy   2.610250      0.008889          NaN
3  2017-06-15  sell   3.809250   2464.606686   775.757161
4  2017-06-16   buy   3.790500      0.781734          NaN
5  2017-07-31  sell   4.062750   2642.407027   177.018607
6  2017-08-01   buy   4.112250      2.342622          NaN
7  2017-08-08  sell   4.257500   2738.083099    93.333450
8  2017-08-10   buy   4.118500      3.398945          NaN
9  2017-09-07  sell   4.164500   2772.064010    30.581966
10 2017-09-18   buy   4.688750      1.012884          NaN
11 2017-10-24  sell   4.967000   2937.582880   164.505987
12 2017-10-26   buy   4.892250      2.232844          NaN
13 2017-11-08  sell   5.229000   3142.019437   202.203712
14 2017-11-14   buy   5.354500      4.282544          NaN
15 2018-03-26  sell   6.112000   3590.802932   444